In [ ]:
import os
import cv2
import h5py
import matplotlib
import numpy as np
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
from numpy.random import default_rng

rng = default_rng(42)

import torch
import torch.nn.functional as F

from f3 import init_event_model, load_weights_ckpt
from f3.utils import setup_torch, plot_patched_features, BaseExtractor, ev_to_frames, smooth_time_weighted_rgb_encoding

setup_torch(cudnn_benchmark=False)

import seaborn as sns

sns.set_theme(
    context="talk",
    style="ticks",
    font_scale=1,
    rc={
        "axes.grid": True,
        "grid.color": ".8",
        "grid.linewidth": 0.75,
        "xtick.direction": "in",
        "ytick.direction": "in",
        "axes.spines.right": False,
        "axes.spines.top": False,
    },
)


In [ ]:
"""
    Functions to handle event subsampling and feature similarity calculations.
"""

def subsample_events(events, factor):
    """
        events: (N, 4)
        factor: float
    """
    num_events = events.shape[0]
    indices = np.sort(rng.choice(num_events, size=int(num_events / factor), replace=False, shuffle=False))
    # indices = np.linspace(0, num_events - 1, int(num_events / factor), dtype=int)
    subsampled_events = events[indices]
    # print(f"Subsampled {num_events} events to {subsampled_events.shape[0]} events")
    return subsampled_events, subsampled_events.shape[0]

def similarity(f1, f2):
    """
        f1: (B, W, H, C)
        f2: (B, W, H, C)
    """
    C = f1.shape[-1]
    f1 = f1.clone()[0].permute(2, 0, 1) # (C, W, H)
    f2 = f2.clone()[0].permute(2, 0, 1) # (C, W, H)
    f1 = f1.reshape(C, -1) # (C, W*H)
    f2 = f2.reshape(C, -1) # (C, W*H)

    B, C = f1.shape[:2]
    normfactor = torch.cat([
        f1.reshape(B, C, -1), f2.reshape(B, C, -1)
    ], dim=-1).norm(dim=-1)[:, :, None, None] + 1e-7
    f1 /= normfactor # (B, C, W, H)
    f2 /= normfactor # (B, C, W, H)

    # L2 distance between f1 and f2
    # dist = torch.mean((f1 - f2) ** 2) / torch.mean(f1 ** 2)
    # return dist.item()

    # Cosine similarity between f1 and f2
    sim = F.cosine_similarity(f1, f2, dim=0)
    sim = torch.mean(sim)
    return sim.item()

    # Bhattacharyya distance between f1 and f2
    # normalize f1 and f2 into probability distributions, where f can be negative
    # f1 = f1.abs()
    # f2 = f2.abs()
    # f1 /= f1.sum(dim=0, keepdim=True)
    # f2 /= f2.sum(dim=0, keepdim=True)
    # bc = torch.sum(torch.sqrt(f1 * f2), dim=0)
    # bc = -torch.log(bc + 1e-7)  # add small value to avoid log(0)
    # return bc.mean().item()   


In [ ]:
config = "/home/richeek/GitHub/f3/outputs/patchff_fullcardaynightm3ed_small_20ms/models/config.yml"
ckpt = "/home/richeek/GitHub/f3/outputs/patchff_fullcardaynightm3ed_small_20ms/models/last.pth"
# config = "/home/richeek/GitHub/f3/outputs/patchff_fullcardaym3ed_small_20ms/models/config.yml"
# ckpt = "/home/richeek/GitHub/f3/outputs/patchff_fullcardaym3ed_small_20ms/models/last.pth"

eff = init_event_model(config, return_feat=True).cuda()
eff = torch.compile(eff)

epoch, loss, acc = load_weights_ckpt(eff, ckpt, strict=False)

print(f"Loaded model from epoch {epoch} with loss {loss} and accuracy {acc}")

In [ ]:
# h5path = "/home/richeek/GitHub/f3/data/car_urban_day_city_hall_data/car_urban_day_city_hall_data.h5"
# tspath = "/home/richeek/GitHub/f3/data/car_urban_day_city_hall_data/50khz_car_urban_day_city_hall_data.npy"
# h5path = "/home/richeek/GitHub/f3/data/car_urban_night_city_hall_data/car_urban_night_city_hall_data.h5"
# tspath = "/home/richeek/GitHub/f3/data/car_urban_night_city_hall_data/50khz_car_urban_night_city_hall_data.npy"

# h5path = "/home/richeek/GitHub/f3/data/car_urban_day_rittenhouse_data/car_urban_day_rittenhouse_data.h5"
# tspath = "/home/richeek/GitHub/f3/data/car_urban_day_rittenhouse_data/50khz_car_urban_day_rittenhouse_data.npy"
# h5path = "/home/richeek/GitHub/f3/data/car_urban_night_rittenhouse_data/car_urban_night_rittenhouse_data.h5"
# tspath = "/home/richeek/GitHub/f3/data/car_urban_night_rittenhouse_data/50khz_car_urban_night_rittenhouse_data.npy"

# h5path = "/home/richeek/GitHub/f3/data/car_urban_day_penno_small_loop_data/car_urban_day_penno_small_loop_data.h5"
# tspath = "/home/richeek/GitHub/f3/data/car_urban_day_penno_small_loop_data/50khz_car_urban_day_penno_small_loop_data.npy"

h5path = "/home/richeek/GitHub/f3/data/car_urban_day_ucity_small_loop_data/car_urban_day_ucity_small_loop_data.h5"
tspath = "/home/richeek/GitHub/f3/data/car_urban_day_ucity_small_loop_data/50khz_car_urban_day_ucity_small_loop_data.npy"
# h5path = "/home/richeek/GitHub/f3/data/car_urban_night_ucity_small_loop_data/car_urban_night_ucity_small_loop_data.h5"
# tspath = "/home/richeek/GitHub/f3/data/car_urban_night_ucity_small_loop_data/50khz_car_urban_night_ucity_small_loop_data.npy"

# h5path = "/home/richeek/GitHub/f3/data/car_urban_day_schuylkill_tunnel_data/car_urban_day_schuylkill_tunnel_data.h5"
# tspath = "/home/richeek/GitHub/f3/data/car_urban_day_schuylkill_tunnel_data/50khz_car_urban_day_schuylkill_tunnel_data.npy"

h5file = h5py.File(h5path, 'r')
tsfile = np.load(tspath, allow_pickle=True)[()]

extractor = BaseExtractor(h5path, tspath, w=1280, h=720, time_ctx=20000,
                          time_pred=20000, bucket=1000, max_numevents_ctx=2000000,
                          randomize_ctx=False, camera="left")

In [ ]:
# time = int(240 * 1e6) # in us
time = int(130 * 1e6) # in us
img_size = (1280, 720)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))
ctx, totcnt = extractor.get_ctx_fixedtime(time)

event_frame = ev_to_frames(ctx, torch.tensor([totcnt], dtype=torch.int32), *img_size)
axes[0].imshow(event_frame[0].T, cmap='gray')
axes[0].set_title(f'Original Events ({totcnt})')
axes[0].axis('off')

sub_events_2, sub_counts_2 = subsample_events(ctx, 2)
event_frame_2 = ev_to_frames(sub_events_2, torch.tensor([sub_counts_2], dtype=torch.int32), *img_size)
axes[1].imshow(event_frame_2[0].T, cmap='gray')
axes[1].set_title(f'Subsampled by 2 ({sub_counts_2})')
axes[1].axis('off')

sub_events_10, sub_counts_10 = subsample_events(ctx, 10)
event_frame_10 = ev_to_frames(sub_events_10, torch.tensor([sub_counts_10], dtype=torch.int32), *img_size)
axes[2].imshow(event_frame_10[0].T, cmap='gray')
axes[2].set_title(f'Subsampled by 10 ({sub_counts_10})')
axes[2].axis('off')

plt.tight_layout()
plt.show()

fig, axes = plt.subplots(1, 3, figsize=(18, 6))
def ret_pca(context, count):
    ctx_tensor = context.cuda()
    cnt_tensor = torch.tensor([count], dtype=torch.int32).cuda()
    
    with torch.no_grad():
        feats = eff(ctx_tensor, cnt_tensor)[1].clone()
    pca, _ = plot_patched_features(feats[0].permute(1, 0, 2), plot=False)
    return pca

pca = ret_pca(ctx, totcnt)
axes[0].imshow(pca)
axes[0].set_title(f'Original Events Features ({totcnt})')
axes[0].axis('off')

pca_2 = ret_pca(sub_events_2, sub_counts_2)
axes[1].imshow(pca_2)
axes[1].set_title(f'Subsampled by 2 Features ({sub_counts_2})')
axes[1].axis('off')

pca_10 = ret_pca(sub_events_10, sub_counts_10)
axes[2].imshow(pca_10)
axes[2].set_title(f'Subsampled by 10 Features ({sub_counts_10})')
axes[2].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
crop = (640, 0, 1280, 640)  # (x1, y1, x2, y2)
event_frame_cropped = event_frame[0, crop[0]:crop[2], crop[1]:crop[3]]
event_frame_10_cropped = event_frame_10[0, crop[0]:crop[2], crop[1]:crop[3]]
plt.imshow(event_frame.T, cmap='gray')
plt.show()
plt.imshow(event_frame_cropped.T, cmap='gray')
plt.show()
plt.imshow(event_frame_10_cropped.T, cmap='gray')
plt.show()

pca_cropped = pca[crop[1]:crop[3], crop[0]:crop[2]]
pca_10_cropped = pca_2[crop[1]:crop[3], crop[0]:crop[2]]
plt.imshow(pca)
plt.show()
plt.imshow(pca_cropped)
plt.show()
plt.imshow(pca_10_cropped)
plt.show()

cv2.imwrite("/home/richeek/GitHub/f3/outputs/matching/event_subsampling.jpg", event_frame.T.numpy())
cv2.imwrite("/home/richeek/GitHub/f3/outputs/matching/event_subsampling_10.jpg", event_frame_10.T.numpy())
cv2.imwrite("/home/richeek/GitHub/f3/outputs/matching/event_subsampling_cropped.jpg", event_frame_cropped.T.numpy())
cv2.imwrite("/home/richeek/GitHub/f3/outputs/matching/event_subsampling_cropped_10.jpg", event_frame_10_cropped.T.numpy())

cv2.imwrite("/home/richeek/GitHub/f3/outputs/matching/event_subsampling_features.jpg",
            cv2.cvtColor(pca, cv2.COLOR_BGR2RGB))
cv2.imwrite("/home/richeek/GitHub/f3/outputs/matching/event_subsampling_features_10.jpg",
            cv2.cvtColor(pca_2, cv2.COLOR_BGR2RGB))
cv2.imwrite("/home/richeek/GitHub/f3/outputs/matching/event_subsampling_features_cropped.jpg",
            cv2.cvtColor(pca_cropped, cv2.COLOR_BGR2RGB))
cv2.imwrite("/home/richeek/GitHub/f3/outputs/matching/event_subsampling_features_cropped_10.jpg",
            cv2.cvtColor(pca_10_cropped, cv2.COLOR_BGR2RGB))

In [ ]:
ctr = 0
time_ctx = 20000 # in us
last_event_time = extractor.events_t[-1] # in us
least_event_count = 400000

# Extract dataset name from h5path
dataset_name = h5path.split('/')[-2]  # Get the folder name which contains the dataset name

# Initialize or load existing similarity_tensors dictionary
try:
    similarity_tensors
except NameError:
    similarity_tensors = {}

similarity_tensors[dataset_name] = []

subsample_fracs = np.arange(1, 22, 1)

for t0 in tqdm(np.arange(time_ctx, last_event_time, 1e6)):
    ctx, totcnt = extractor.get_ctx_fixedtime(int(t0))
    if totcnt < least_event_count: continue
    with torch.no_grad():
        feat_0 = eff(ctx.cuda(), torch.tensor([totcnt], dtype=torch.int32).cuda())[1]
    similarity_tensors[dataset_name].append(torch.zeros(subsample_fracs.size, dtype=torch.float32))
    for i, frac in enumerate(subsample_fracs):
        sub_events, sub_counts = subsample_events(ctx, frac)
        with torch.no_grad():
            feat = eff(sub_events.cuda(), torch.tensor([sub_counts], dtype=torch.int32).cuda())[1]
        dist = similarity(feat_0, feat)
        # print(f"Distance between {i} subsampled events and original events: {dist}")
        similarity_tensors[dataset_name][ctr][i] = dist
    ctr += 1

similarity_tensors[dataset_name] = torch.stack(similarity_tensors[dataset_name], dim=0)
similarity_tensors[dataset_name] = similarity_tensors[dataset_name].cpu().numpy()

In [ ]:
# matplotlib.rcParams['font.family'] = 'sans-serif'
# matplotlib.rcParams['text.usetex'] = True

plt.figure(figsize=(8, 6))

# Use seaborn color palette
colors = sns.color_palette("husl", len(similarity_tensors))

for i, (dataset_name, similarity_data) in enumerate(similarity_tensors.items()):
    similarity_mean = np.mean(similarity_data, axis=0)
    similarity_std = np.std(similarity_data, axis=0)
    
    color = colors[i]
    
    plt.plot(subsample_fracs, similarity_mean, linewidth=2, color=color, 
             label=dataset_name.replace('_', ' ').title(), alpha=0.8)
    plt.fill_between(subsample_fracs, similarity_mean - similarity_std, 
                     similarity_mean + similarity_std, color=color, alpha=0.2)

plt.xlabel("Subsampling Factor", fontsize=14)
plt.ylabel("Cosine Similarity", fontsize=14)
plt.grid(True, alpha=0.3)
plt.legend(loc='upper right', fontsize=10)
plt.title('Feature similarity with event sub-sampling', fontsize=16, pad=10)
plt.xticks(np.arange(1, 22, 2), fontsize=12)
plt.yticks(fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
# save the similarity tensors
output_dir = "/home/richeek/GitHub/f3/outputs/matching/"
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, "similarity_tensors.npy")
np.save(output_path, similarity_tensors)
print(f"Saved similarity tensors to {output_path}")

In [ ]:
# matplotlib.rcParams['font.family'] = 'sans-serif'
# matplotlib.rcParams['text.usetex'] = True

plt.figure(figsize=(10, 7))

output_path = "/home/richeek/GitHub/f3/outputs/matching/similarity_tensors.npy"
similarity_tensors_saved = np.load(output_path, allow_pickle=True).item()
plotlist = [
    "car_urban_day_ucity_small_loop_data",
    "car_urban_night_ucity_small_loop_data",
    "car_urban_day_rittenhouse_data",
    "car_urban_night_rittenhouse_data",
]

# try:
#     subsample_fracs
# except NameError:
#     subsample_fracs = np.arange(1, 21, 1)
subsample_fracs = np.arange(1, 21, 1)

# colors = sns.color_palette("husl")
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']  # Custom colors
markers = ['o', 's', '^', 'D']

for idx, dataset_name in enumerate(plotlist):
    similarity_data = similarity_tensors_saved[dataset_name][:, :len(subsample_fracs)]
    similarity_mean = np.mean(similarity_data, axis=0)
    similarity_std = np.std(similarity_data, axis=0)
    
    color = colors[idx]
    marker = markers[idx]
    
    plt.plot(subsample_fracs, similarity_mean, linewidth=3, label=dataset_name.replace('_', ' ').replace(" data", "").replace("car urban", "").title(), 
             color=color, marker=marker, markersize=6, markevery=2)
    plt.fill_between(subsample_fracs, similarity_mean - similarity_std, 
                     similarity_mean + similarity_std, color=color, alpha=0.2)

plt.xlabel("Sub-sampling Factor", fontsize=20)
plt.ylabel("Cosine Similarity", fontsize=20)
plt.grid(True, alpha=0.3)
# plt.legend(loc='upper right', fontsize=20)
# plt.title('Feature similarity with event sub-sampling', fontsize=24, pad=15)
plt.xticks([1, 4, 8, 12, 16, 20], fontsize=16)
plt.yticks(fontsize=16)
plt.tight_layout()
# plt.savefig(os.path.join(output_dir, "feature_similarity_with_subsampling.pdf"))
plt.savefig("feature_similarity_with_subsampling.svg")
plt.show()

In [ ]:
# Create subplots showing cosine similarity and optical flow AEE
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 7))

# Left subplot: Cosine Similarity (from previous cell)
plotlist = [
    "car_urban_day_ucity_small_loop_data",
    "car_urban_night_ucity_small_loop_data", 
    "car_urban_day_rittenhouse_data",
    "car_urban_night_rittenhouse_data",
]

colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']
markers = ['o', 's', '^', 'D']
labels = ['Day Ucity Small Loop', 'Night Ucity Small Loop', 'Day Rittenhouse', 'Night Rittenhouse']

for idx, dataset_name in enumerate(plotlist):
    similarity_data = similarity_tensors_saved[dataset_name][:, :len(subsample_fracs)]
    similarity_mean = np.mean(similarity_data, axis=0)
    similarity_std = np.std(similarity_data, axis=0)
    
    color = colors[idx]
    marker = markers[idx]
    label = labels[idx]
    
    ax1.plot(subsample_fracs, similarity_mean, linewidth=3, 
             label=label, color=color, marker=marker, markersize=6, markevery=2)
    ax1.fill_between(subsample_fracs, similarity_mean - similarity_std, 
                     similarity_mean + similarity_std, color=color, alpha=0.2)

ax1.set_xlabel("Sub-sampling Factor", fontsize=20)
ax1.set_ylabel("Cosine Similarity", fontsize=20)
ax1.grid(True, alpha=0.3)
ax1.set_xticks([1, 4, 8, 12, 16, 20])
ax1.tick_params(axis='both', which='major', labelsize=16)

# Right subplot: Optical Flow AEE
subsampling_factors = [1, 2, 5, 10]
day_ucity_small_loop = [5.41, 5.82, 7.12, 9.11]
night_ucity_small_loop = [6.97, 7.79, 9.85, 12.83]
day_rittenhouse = [5.31, 5.78, 6.57, 8.23]
night_rittenhouse = [6.15, 7.77, 10.08, 12.14]

ax2.plot(subsampling_factors, day_ucity_small_loop, 
         linewidth=3, marker=markers[0], color=colors[0], label=labels[0])
ax2.plot(subsampling_factors, night_ucity_small_loop, 
         linewidth=3, marker=markers[1], color=colors[1], label=labels[1])
ax2.plot(subsampling_factors, day_rittenhouse, 
         linewidth=3, marker=markers[2], color=colors[2], label=labels[2])
ax2.plot(subsampling_factors, night_rittenhouse, 
         linewidth=3, marker=markers[3], color=colors[3], label=labels[3])

ax2.set_xlabel('Subsampling Factor', fontsize=20)
ax2.set_ylabel('Average Endpoint Error (AEE)', fontsize=20)
ax2.grid(True, alpha=0.3)
ax2.set_xticks(subsampling_factors)
ax2.set_yticks([4, 6, 8, 10, 12])
ax2.tick_params(axis='both', which='major', labelsize=16)
ax2.set_ylim(4, 13)

# Create a single legend for both subplots
handles, labels = ax1.get_legend_handles_labels()
fig.legend(handles, labels, loc='upper center', 
           ncol=2, fontsize=20)

plt.tight_layout()
plt.subplots_adjust(bottom=0.15)  # Make room for the legend
plt.savefig('combined_subsampling_analysis.svg', bbox_inches='tight')
plt.show()


## Nighttime Qualitative results on Flow and Depth.

In [ ]:
import yaml
from f3.tasks.optical_flow.utils import flow_viz_np, EventFFFlow


# h5path = "/home/richeek/GitHub/f3/data/car_urban_night_city_hall_data/car_urban_night_city_hall_data.h5"
# tspath = "/home/richeek/GitHub/f3/data/car_urban_night_city_hall_data/50khz_car_urban_night_city_hall_data.npy"
# h5path = "/home/richeek/GitHub/f3/data/car_urban_night_rittenhouse_data/car_urban_night_rittenhouse_data.h5"
# tspath = "/home/richeek/GitHub/f3/data/car_urban_night_rittenhouse_data/50khz_car_urban_night_rittenhouse_data.npy"
# h5path = "/home/richeek/GitHub/f3/data/car_urban_night_penno_small_loop_darker_data/car_urban_night_penno_small_loop_darker_data.h5"
# tspath = "/home/richeek/GitHub/f3/data/car_urban_night_penno_small_loop_darker_data/50khz_car_urban_night_penno_small_loop_darker_data.npy"

# SPOT
h5path = "/home/richeek/GitHub/f3/data/spot_indoor_building_loop/spot_indoor_building_loop_data.h5"
tspath = "/home/richeek/GitHub/f3/data/spot_indoor_building_loop/50khz_spot_indoor_building_loop_data.npy"


h5file = h5py.File(h5path, 'r')
tsfile = np.load(tspath, allow_pickle=True)[()]

extractor = BaseExtractor(h5path, tspath, w=1280, h=720, time_ctx=20000,
                          time_pred=20000, bucket=1000, max_numevents_ctx=2000000,
                          randomize_ctx=False, camera="left")

config = "/home/richeek/GitHub/f3/outputs/optical_flow/optflow_trainm3ed_20msff_pyr5_28k/models/flow_config.yaml"
ckpt_path = "/home/richeek/GitHub/f3/outputs/optical_flow/optflow_trainm3ed_20msff_pyr5_28k/models/last.pth"

flow_yaml = yaml.safe_load(open(config, 'r'))
flow_model = EventFFFlow(flow_yaml["eventff"], flowhead_config=flow_yaml["flowhead"])
flow_model.eventff = torch.compile(flow_model.eventff)
flow_model.flowhead = torch.compile(flow_model.flowhead)

ckpt = torch.load(ckpt_path)
flow_model.load_state_dict(ckpt["model"])
epoch, loss = ckpt["epoch"], ckpt["loss"]
del ckpt
torch.cuda.empty_cache()
print(f"Loaded optical flow model from {ckpt_path}. Epoch: {epoch}, Loss: {loss}")

In [ ]:
rgb_idx = 500
time = h5file["ovc/ts"][rgb_idx] # in us

rgb = h5file["ovc/rgb/data"][rgb_idx]
plt.imshow(rgb)
plt.show()

ctx, totcnt = extractor.get_ctx_fixedtime(time)
eframe = ev_to_frames(ctx, torch.tensor([totcnt], dtype=torch.int32), *img_size)
plt.imshow(eframe[0].T, cmap='gray')
plt.show()

with torch.no_grad():
    flow, _ = flow_model(ctx.cuda(), torch.tensor([totcnt], dtype=torch.int32).cuda())
flow_rgb = flow_viz_np(flow[0].permute(1, 2, 0).cpu().numpy())
plt.imshow(flow_rgb)
plt.show()

In [ ]:
from matplotlib import colormaps as cm
from f3.tasks.depth.utils import EventFFDepthAnythingV2, get_disparity_image

config = {
    "dav2_config": {"size": 518, "encoder": "vitb"},
    "eventff": "/home/richeek/GitHub/f3/outputs/patchff_fullcardaym3ed_small_20ms/models/config.yml",
}
ckpt_path = "/home/richeek/GitHub/f3/outputs/monoculardepth/dav2b_fullm3ed_pseudo_518x518x20/models/best.pth"

depth_model = EventFFDepthAnythingV2(config["eventff"], config["dav2_config"])
depth_model.eventff = torch.compile(depth_model.eventff)

last_dict = torch.load(ckpt_path, weights_only=True)
depth_model.load_state_dict(last_dict["model"], strict=False)
epoch, results = last_dict["epoch"], last_dict["results"]
del last_dict
torch.cuda.empty_cache()
print(f"Loaded depth model from {ckpt_path}. Epoch: {epoch}, Results: {results}")

In [ ]:
rgb_idx = 500
time = h5file["ovc/ts"][rgb_idx] # in us
cmap = cm.get_cmap("magma")

rgb = h5file["ovc/rgb/data"][rgb_idx]
plt.imshow(rgb)
plt.show()

ctx, totcnt = extractor.get_ctx_fixedtime(time)
eframe = ev_to_frames(ctx, torch.tensor([totcnt], dtype=torch.int32), *img_size)
plt.imshow(eframe[0].T, cmap='gray')
plt.show()

depth_pred, ff = depth_model.infer_image(ctx.cuda(), torch.tensor([totcnt], dtype=torch.int32).cuda())
depth_image = get_disparity_image(depth_pred, torch.ones_like(depth_pred, dtype=bool), cmap=cmap)
plt.imshow(depth_image)
plt.show()

In [ ]:
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('depth_flow_night_video_penno_small_loop_darker.mp4', fourcc, 25.0, (1920, 1080))
down_size = (1920 // 2, 1080 // 2)

for rgb_idx in tqdm(range(0, len(h5file["ovc/rgb/data"]))):
    time = h5file["ovc/ts"][rgb_idx]  # in us
    if time <= 20000: continue  # skip first 20ms

    rgb = h5file["ovc/rgb/data"][rgb_idx]
    ctx, totcnt = extractor.get_ctx_fixedtime(time)

    if totcnt < 50000: continue  # skip frames with less than 100k events
    eframe = ev_to_frames(ctx, torch.tensor([totcnt], dtype=torch.int32), *img_size)[0].T.cpu().numpy()[..., None].repeat(3, axis=-1)

    with torch.no_grad():
        flow, _ = flow_model(ctx.cuda(), torch.tensor([totcnt], dtype=torch.int32).cuda())
        depth_pred, ff = depth_model.infer_image(ctx.cuda(), torch.tensor([totcnt], dtype=torch.int32).cuda())
    flow_rgb = flow_viz_np(flow[0].permute(1, 2, 0).cpu().numpy()) * (eframe == 255)
    depth_rgb = get_disparity_image(depth_pred, torch.ones_like(depth_pred, dtype=bool), cmap=cmap)

    # place on 4 quadrants
    combined = np.zeros((down_size[1] * 2, down_size[0] * 2, 3), dtype=np.uint8)
    combined[:down_size[1], :down_size[0]] = cv2.resize(rgb, down_size)
    combined[:down_size[1], down_size[0]:] = cv2.resize(eframe, down_size)
    combined[down_size[1]:, :down_size[0]] = cv2.resize(flow_rgb, down_size)
    combined[down_size[1]:, down_size[0]:] = cv2.resize(depth_rgb, down_size)
    combined = cv2.cvtColor(combined, cv2.COLOR_RGB2BGR)
    out.write(combined)
out.release()

# Denoising by thresholding

In [ ]:
config = "/home/richeek/GitHub/f3/outputs/patchff_fullcardaym3ed_small_20ms/models/config.yml"
ckpt = "/home/richeek/GitHub/f3/outputs/patchff_fullcardaym3ed_small_20ms/models/last.pth"

# config = "/home/richeek/GitHub/f3/outputs/patchff_fullcardaynightm3ed_small_20ms/models/config.yml"
# ckpt = "/home/richeek/GitHub/f3/outputs/patchff_fullcardaynightm3ed_small_20ms/models/last.pth"

# h5path = "/home/richeek/GitHub/f3/data/car_urban_day_city_hall_data/car_urban_day_city_hall_data.h5"
# tspath = "/home/richeek/GitHub/f3/data/car_urban_day_city_hall_data/50khz_car_urban_day_city_hall_data.npy"
# h5path = "/home/richeek/GitHub/f3/data/car_urban_night_city_hall_data/car_urban_night_city_hall_data.h5"
# tspath = "/home/richeek/GitHub/f3/data/car_urban_night_city_hall_data/50khz_car_urban_night_city_hall_data.npy"

# h5path = "/home/richeek/GitHub/f3/data/car_urban_day_rittenhouse_data/car_urban_day_rittenhouse_data.h5"
# tspath = "/home/richeek/GitHub/f3/data/car_urban_day_rittenhouse_data/50khz_car_urban_day_rittenhouse_data.npy"
# h5path = "/home/richeek/GitHub/f3/data/car_urban_night_rittenhouse_data/car_urban_night_rittenhouse_data.h5"
# tspath = "/home/richeek/GitHub/f3/data/car_urban_night_rittenhouse_data/50khz_car_urban_night_rittenhouse_data.npy"

# h5path = "/home/richeek/GitHub/f3/data/car_urban_day_penno_small_loop_data/car_urban_day_penno_small_loop_data.h5"
# tspath = "/home/richeek/GitHub/f3/data/car_urban_day_penno_small_loop_data/50khz_car_urban_day_penno_small_loop_data.npy"

# h5path = "/home/richeek/GitHub/f3/data/car_urban_day_ucity_small_loop_data/car_urban_day_ucity_small_loop_data.h5"
# tspath = "/home/richeek/GitHub/f3/data/car_urban_day_ucity_small_loop_data/50khz_car_urban_day_ucity_small_loop_data.npy"
# h5path = "/home/richeek/GitHub/f3/data/car_urban_night_ucity_small_loop_data/car_urban_night_ucity_small_loop_data.h5"
# tspath = "/home/richeek/GitHub/f3/data/car_urban_night_ucity_small_loop_data/50khz_car_urban_night_ucity_small_loop_data.npy"

# h5path = "/home/richeek/GitHub/f3/data/car_urban_day_schuylkill_tunnel_data/car_urban_day_schuylkill_tunnel_data.h5"
# tspath = "/home/richeek/GitHub/f3/data/car_urban_day_schuylkill_tunnel_data/50khz_car_urban_day_schuylkill_tunnel_data.npy"

# SPOT
# h5path = "/home/richeek/GitHub/f3/data/spot_indoor_building_loop/spot_indoor_building_loop_data.h5"
# tspath = "/home/richeek/GitHub/f3/data/spot_indoor_building_loop/50khz_spot_indoor_building_loop_data.npy"
h5path = "/home/richeek/GitHub/f3/data/spot_indoor_obstacles/spot_indoor_obstacles_data.h5"
tspath = "/home/richeek/GitHub/f3/data/spot_indoor_obstacles/50khz_spot_indoor_obstacles_data.npy"
# h5path = "/home/richeek/GitHub/f3/data/spot_indoor_stairwell/spot_indoor_stairwell_data.h5"
# tspath = "/home/richeek/GitHub/f3/data/spot_indoor_stairwell/50khz_spot_indoor_stairwell_data.npy"

h5file = h5py.File(h5path, 'r')
tsfile = np.load(tspath, allow_pickle=True)[()]

extractor = BaseExtractor(h5path, tspath, w=1280, h=720, time_ctx=20000,
                          time_pred=20000, bucket=1000, max_numevents_ctx=2000000,
                          randomize_ctx=False, camera="left")

eff = init_event_model(config, return_feat=True).cuda()
eff = torch.compile(eff)

epoch, loss, acc = load_weights_ckpt(eff, ckpt, strict=False)

print(f"Loaded model from epoch {epoch} with loss {loss} and accuracy {acc}")

In [ ]:
from IPython.display import display, clear_output
import time as time_module

start_idx = 200
end_idx = len(h5file["ovc/rgb/data"])  # You can adjust this to a smaller range if needed
step = 10

plt.figure(figsize=(16, 10))
for rgb_idx in range(start_idx, end_idx, step):
    clear_output(wait=True)
    timestamp = h5file["ovc/ts"][rgb_idx]  # in us
    rgb = h5file["ovc/rgb/data"][rgb_idx]
    ctx, totcnt = extractor.get_ctx_fixedtime(timestamp)
    eframe = ev_to_frames(ctx, torch.tensor([totcnt], dtype=torch.int32), *img_size)
    
    ctx_tensor = ctx.cuda()
    cnt_tensor = torch.tensor([totcnt], dtype=torch.int32).cuda()
    
    with torch.no_grad():
        logits, feats = eff(ctx_tensor, cnt_tensor)
    
    # # Get PCA visualization of features
    # pca, _ = plot_patched_features(feats[0].permute(1, 0, 2), plot=False)

    # with torch.no_grad():
    #     flow, _ = flow_model(ctx_tensor, cnt_tensor)
    # flow_rgb = flow_viz_np(flow[0].permute(1, 2, 0).cpu().numpy()) * (eframe[0].T.cpu().numpy() == 255)[..., None]

    with torch.no_grad():
        depth_pred, ff = depth_model.infer_image(ctx_tensor, cnt_tensor)
    depth_image = get_disparity_image(depth_pred, torch.ones_like(depth_pred, dtype=bool), cmap=cm.get_cmap("magma"))
    
    predframe = smooth_time_weighted_rgb_encoding(torch.sigmoid(logits).cpu().numpy() > 0.6225)
    
    plt.figure(figsize=(16, 10))
    
    plt.subplot(2, 2, 1)
    plt.imshow(rgb[40:-40,:,::-1])
    plt.axis('off')
    plt.title(f"RGB Frame (Index: {rgb_idx})")
    
    plt.subplot(2, 2, 2)
    plt.title(f"Event Frame (Events: {totcnt})")
    plt.imshow(eframe[0].T, cmap='gray')
    plt.axis('off')
    
    # plt.subplot(2, 2, 3)
    # plt.title("PCA of Features")
    # plt.imshow(pca)
    # plt.axis('off')

    # plt.subplot(2, 2, 3)
    # plt.title("Optical Flow")
    # plt.imshow(flow_rgb)
    # plt.axis('off')

    plt.subplot(2, 2, 3)
    plt.title("Depth Prediction")
    plt.imshow(depth_image)
    plt.axis('off')
    
    plt.subplot(2, 2, 4)
    plt.title("Predicted Frame")
    plt.imshow(predframe[0].transpose(1, 0, 2))
    plt.axis('off')
    
    plt.tight_layout()
    plt.suptitle(f"Timestamp: {timestamp/1e6:.2f} seconds", fontsize=16)
    display(plt.gcf())
    
    time_module.sleep(0.04)
    plt.close()
print("Animation complete!")